In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
import torch.nn as nn

# Construct customized ResNet

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, pcpt_block, pcpt_layers, scoop_block, scoop_layers, h, w, pcpt_is_upsample=0, scoop_is_upsample=0):
        self.inplanes = 64
        self.pcpt_is_upsample = pcpt_is_upsample
        super(ResNet, self).__init__()
        self.pcpt_conv1 = nn.Conv2d(4, 64, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.pcpt_bn1 = nn.BatchNorm2d(64)
        self.pcpt_relu = nn.ReLU(inplace=True)
        self.pcpt_maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.pcpt_upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        self.pcpt_layer1 = self._make_layer(pcpt_block, 128, pcpt_layers[0])
        self.pcpt_layer2 = self._make_layer(pcpt_block, 256, pcpt_layers[1])
        self.pcpt_layer3 = self._make_layer(pcpt_block, 512, pcpt_layers[2])

        self.inplanes = 512
        self.scoop_is_upsample = scoop_is_upsample
        self.scoop_upsample = nn.Upsample(scale_factor=2, mode='bilinear')
        self.scoop_layer1 = self._make_layer(scoop_block, 256, scoop_layers[0])
        self.scoop_layer2 = self._make_layer(scoop_block, 128, scoop_layers[1])
        self.scoop_layer3 = self._make_layer(scoop_block, 64, scoop_layers[2])
        self.scoop_conv1 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.scoop_bn1 = nn.BatchNorm2d(1)
        self.scoop_conv2 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.scoop_bn2 = nn.BatchNorm2d(3)
        self.scoop_relu = nn.ReLU(inplace=True)

        self.scoop_head = nn.Linear(h*w, 1)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.pcpt_conv1(x)
        x = self.pcpt_bn1(x)
        x = self.pcpt_relu(x)
        x = self.pcpt_maxpool(x)

        x = self.pcpt_layer1(x)
        x = self.pcpt_maxpool(x)
        x = self.pcpt_layer2(x)
        x = self.pcpt_layer3(x)

        x = self.scoop_layer1(x)
        x = self.scoop_layer2(x)
        x = self.scoop_upsample(x)
        x = self.scoop_layer3(x)
        x = self.scoop_upsample(x)

        x = self.scoop_conv2(x)
        x = self.scoop_bn2(x)
        x = self.scoop_relu(x)

        return x

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.utils.data as Data
import scipy.ndimage
BATCH_SIZE =  32
NUM_EPOCH = 50

# make data

data_input = np.load("data_20210503/input_data_array.npy")/255.0
#normalization=np.ones(data_input.shape)
#normalization[:,:,:]=[255,255,255,0.08]
#data_input = data_input/normalization
print(data_input.shape)

(11016, 200, 200, 4)


In [4]:
'''
input_array_mean = np.mean(np.mean(np.mean(data_input, axis=0), axis=0), axis=0)
input_array_std = np.mean(np.mean(np.std(data_input, axis=0), axis=0), axis=0)
print('input_array_mean', input_array_mean)
print('input_array_std', input_array_std)
'''

#input_array_mean=np.ones(data_input.shape)
#input_array_mean[:,:,:]=[0.54569177,0.48216905,0.49853667,0.42829879]
#input_array_std=np.ones(data_input.shape)
#input_array_std[:,:,:]=[0.31950833,0.29141234,0.30389949,0.20277719]

#data_input = (data_input-input_array_mean)/input_array_std


data_input = torch.from_numpy(data_input).permute(0,3,1,2)
train_data_input = data_input[0: data_input.shape[0]-30, :, :, :]
val_data_input = data_input[data_input.shape[0]-30: data_input.shape[0], :, :, :]

data_label = np.load("data_20210503/label_data_array.npy")

data_label[data_label==0]=2
data_label[data_label==255]=0
data_label[(data_label==128)]=1
#print(data_label[np.logical_and(np.logical_and((data_label!=0),(data_label!=1)),(data_label!=2))])


good_cnt = (data_label==1).sum()
bad_cnt = (data_label==2).sum()
print(good_cnt, bad_cnt)
total = good_cnt + bad_cnt

weights = [0, total/good_cnt, total/bad_cnt]
class_weights = torch.FloatTensor(weights)
if torch.cuda.is_available():
    class_weights = class_weights.cuda()
print('Class weights:', class_weights)

data_label = torch.from_numpy(data_label).long()
train_data_label = data_label[0: data_label.shape[0]-30, :, :]
val_data_label = data_label[data_label.shape[0]-30: data_label.shape[0], :, :]

train_torch_dataset = Data.TensorDataset(train_data_input, train_data_label)
loader = Data.DataLoader(dataset=train_torch_dataset, batch_size=BATCH_SIZE, shuffle=True)
htmap_h = data_input.shape[2]
htmap_w = data_input.shape[3]



406230 2018235
Class weights: tensor([0.0000, 5.9682, 1.2013], device='cuda:0')


In [5]:
net = ResNet(pcpt_block=BasicBlock, pcpt_layers=[1,5,1], scoop_block=BasicBlock, scoop_layers=[1,5,1], h=htmap_h, w=htmap_w).cuda()     # define the network
print(net)  # net architecture

ResNet(
  (pcpt_conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (pcpt_bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pcpt_relu): ReLU(inplace=True)
  (pcpt_maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pcpt_upsample): Upsample(scale_factor=2.0, mode=bilinear)
  (pcpt_layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum

In [7]:
optimizer = torch.optim.SGD(params = net.parameters(), lr=1e-4, momentum=0.9, weight_decay=2e-5)
#optimizer = torch.optim.Adam(params = net.parameters(), lr=1e-4)
#optimizer = torch.optim.SGD(params = net.parameters(), lr=1e-4)
loss_func = torch.nn.CrossEntropyLoss(ignore_index = 0, weight=class_weights).cuda()  # the target label is NOT an one-hotted

val_data_input = val_data_input.cuda().float()
val_data_label = val_data_label.cuda()

min_val_loss_print=float('inf')
for epoch in range(NUM_EPOCH):
    for step, (batch_data, batch_label) in enumerate(loader):
        batch_data = batch_data.cuda().float()
        batch_label = batch_label.cuda()
        net.train()
        output  = net(batch_data)
        train_loss = loss_func(output,batch_label)
        optimizer.zero_grad()
        train_loss.backward()
        #nn.utils.clip_grad_norm_(net.parameters(), max_norm=20, norm_type=2)
        optimizer.step()
        train_loss_print = train_loss.data.item()
        
        with torch.no_grad():
            net.eval()
            val_output  = net(val_data_input)
            val_loss = loss_func(val_output,val_data_label)
            val_loss_print = val_loss.data.item()
            torch.cuda.empty_cache()
        #print('epoch: ', epoch, '  step: ', step, '  train loss: ', train_loss_print, ' val loss: ', val_loss_print)
        print('epoch: ', epoch, '  step: ', step, '  train loss: ', train_loss_print, ' val loss: ', val_loss_print)
        if val_loss_print < min_val_loss_print:
            torch.save(net.state_dict(), 'net_params_20210503/net_params_20210503_01/epoch_'+str(epoch)+'.pkl') 
            min_val_loss_print = val_loss_print
            print('min_val_loss_print', min_val_loss_print)

epoch:  0   step:  0   train loss:  1.2335237264633179  val loss:  1.1043323278427124
min_val_loss_print 1.1043323278427124
epoch:  0   step:  1   train loss:  1.2284549474716187  val loss:  1.0534299612045288
min_val_loss_print 1.0534299612045288
epoch:  0   step:  2   train loss:  1.2841556072235107  val loss:  1.079496145248413
epoch:  0   step:  3   train loss:  1.1853079795837402  val loss:  1.168154239654541
epoch:  0   step:  4   train loss:  1.1608363389968872  val loss:  1.1972343921661377
epoch:  0   step:  5   train loss:  1.1357402801513672  val loss:  1.2147738933563232
epoch:  0   step:  6   train loss:  1.144417405128479  val loss:  1.2305231094360352
epoch:  0   step:  7   train loss:  1.168941855430603  val loss:  1.2437999248504639
epoch:  0   step:  8   train loss:  1.1132926940917969  val loss:  1.3085803985595703
epoch:  0   step:  9   train loss:  1.13316011428833  val loss:  1.3532899618148804
epoch:  0   step:  10   train loss:  1.1312527656555176  val loss:  1.

epoch:  0   step:  94   train loss:  0.7787453532218933  val loss:  1.0946744680404663
epoch:  0   step:  95   train loss:  0.7232161164283752  val loss:  1.0896122455596924
epoch:  0   step:  96   train loss:  0.8108125925064087  val loss:  1.0879814624786377
epoch:  0   step:  97   train loss:  0.7937119007110596  val loss:  1.0844051837921143
epoch:  0   step:  98   train loss:  0.7641717195510864  val loss:  1.0863267183303833
epoch:  0   step:  99   train loss:  0.7214170098304749  val loss:  1.0881752967834473
epoch:  0   step:  100   train loss:  0.6882071495056152  val loss:  1.0836213827133179
epoch:  0   step:  101   train loss:  0.7453848719596863  val loss:  1.0783249139785767
epoch:  0   step:  102   train loss:  0.751952588558197  val loss:  1.0776264667510986
epoch:  0   step:  103   train loss:  0.8310127854347229  val loss:  1.074295163154602
epoch:  0   step:  104   train loss:  0.8630310893058777  val loss:  1.0732964277267456
epoch:  0   step:  105   train loss:  0.

epoch:  0   step:  183   train loss:  0.5607079267501831  val loss:  1.0433772802352905
epoch:  0   step:  184   train loss:  0.5704908967018127  val loss:  1.0391621589660645
epoch:  0   step:  185   train loss:  0.5660005211830139  val loss:  1.0391632318496704
epoch:  0   step:  186   train loss:  0.6284170746803284  val loss:  1.0337727069854736
epoch:  0   step:  187   train loss:  0.5351576805114746  val loss:  1.0353777408599854
epoch:  0   step:  188   train loss:  0.5767691135406494  val loss:  1.036331295967102
epoch:  0   step:  189   train loss:  0.6438424587249756  val loss:  1.0349843502044678
epoch:  0   step:  190   train loss:  0.5485089421272278  val loss:  1.0361069440841675
epoch:  0   step:  191   train loss:  0.5575436353683472  val loss:  1.0390796661376953
epoch:  0   step:  192   train loss:  0.5337405204772949  val loss:  1.0393612384796143
epoch:  0   step:  193   train loss:  0.5300801396369934  val loss:  1.0385831594467163
epoch:  0   step:  194   train lo

epoch:  0   step:  266   train loss:  0.5514103770256042  val loss:  0.939519464969635
epoch:  0   step:  267   train loss:  0.5124726891517639  val loss:  0.9358761310577393
epoch:  0   step:  268   train loss:  0.4779519736766815  val loss:  0.9362013339996338
epoch:  0   step:  269   train loss:  0.4966558516025543  val loss:  0.9371048212051392
epoch:  0   step:  270   train loss:  0.5358237028121948  val loss:  0.9351515769958496
epoch:  0   step:  271   train loss:  0.46594396233558655  val loss:  0.9308488368988037
epoch:  0   step:  272   train loss:  0.586431622505188  val loss:  0.9219648838043213
min_val_loss_print 0.9219648838043213
epoch:  0   step:  273   train loss:  0.49211710691452026  val loss:  0.9211357831954956
min_val_loss_print 0.9211357831954956
epoch:  0   step:  274   train loss:  0.49378183484077454  val loss:  0.9165507555007935
min_val_loss_print 0.9165507555007935
epoch:  0   step:  275   train loss:  0.4240872859954834  val loss:  0.9126670956611633
min_v

epoch:  1   step:  1   train loss:  0.34351441264152527  val loss:  0.7791616916656494
min_val_loss_print 0.7791616916656494
epoch:  1   step:  2   train loss:  0.35779452323913574  val loss:  0.7816834449768066
epoch:  1   step:  3   train loss:  0.4037347733974457  val loss:  0.7843561172485352
epoch:  1   step:  4   train loss:  0.41448554396629333  val loss:  0.7860268950462341
epoch:  1   step:  5   train loss:  0.40600019693374634  val loss:  0.7907997369766235
epoch:  1   step:  6   train loss:  0.3702530860900879  val loss:  0.7888341546058655
epoch:  1   step:  7   train loss:  0.3423885107040405  val loss:  0.7900857329368591
epoch:  1   step:  8   train loss:  0.34402966499328613  val loss:  0.789971113204956
epoch:  1   step:  9   train loss:  0.3425004482269287  val loss:  0.7899829149246216
epoch:  1   step:  10   train loss:  0.4193994104862213  val loss:  0.7942352890968323
epoch:  1   step:  11   train loss:  0.33250826597213745  val loss:  0.7916088104248047
epoch:  1

...

epoch:  12   step:  223   train loss:  0.024872466921806335  val loss:  0.02915174886584282
epoch:  12   step:  224   train loss:  0.01987786591053009  val loss:  0.029452841728925705
epoch:  12   step:  225   train loss:  0.018992891535162926  val loss:  0.029540428891777992
epoch:  12   step:  226   train loss:  0.02170330286026001  val loss:  0.029439685866236687
epoch:  12   step:  227   train loss:  0.04164576530456543  val loss:  0.029244525358080864
epoch:  12   step:  228   train loss:  0.022601187229156494  val loss:  0.02874506637454033
epoch:  12   step:  229   train loss:  0.03669235110282898  val loss:  0.027362514287233353
epoch:  12   step:  230   train loss:  0.017719300463795662  val loss:  0.027018090710043907
epoch:  12   step:  231   train loss:  0.03247735649347305  val loss:  0.027430115267634392
epoch:  12   step:  232   train loss:  0.044291045516729355  val loss:  0.02901167795062065
epoch:  12   step:  233   train loss:  0.04584747552871704  val loss:  0.02868

epoch:  12   step:  312   train loss:  0.011484595015645027  val loss:  0.029445171356201172
epoch:  12   step:  313   train loss:  0.024734610691666603  val loss:  0.03030155785381794
epoch:  12   step:  314   train loss:  0.0389336533844471  val loss:  0.030171768739819527
epoch:  12   step:  315   train loss:  0.023389142006635666  val loss:  0.03048050031065941
epoch:  12   step:  316   train loss:  0.022109324112534523  val loss:  0.030566811561584473
epoch:  12   step:  317   train loss:  0.03915831074118614  val loss:  0.03078591823577881
epoch:  12   step:  318   train loss:  0.02589023858308792  val loss:  0.030848143622279167
epoch:  12   step:  319   train loss:  0.03338796645402908  val loss:  0.030776213854551315
epoch:  12   step:  320   train loss:  0.025843048468232155  val loss:  0.0301055908203125
epoch:  12   step:  321   train loss:  0.04577164724469185  val loss:  0.03047819808125496
epoch:  12   step:  322   train loss:  0.037444744259119034  val loss:  0.03045998

epoch:  13   step:  58   train loss:  0.03277406841516495  val loss:  0.02718736045062542
epoch:  13   step:  59   train loss:  0.015542629174888134  val loss:  0.027188483625650406
epoch:  13   step:  60   train loss:  0.03346072509884834  val loss:  0.027569010853767395
epoch:  13   step:  61   train loss:  0.018128909170627594  val loss:  0.027227619662880898
epoch:  13   step:  62   train loss:  0.0346185527741909  val loss:  0.026693159714341164
epoch:  13   step:  63   train loss:  0.020919103175401688  val loss:  0.025981634855270386
epoch:  13   step:  64   train loss:  0.06507854163646698  val loss:  0.025592556223273277
epoch:  13   step:  65   train loss:  0.026443084701895714  val loss:  0.025912700220942497
epoch:  13   step:  66   train loss:  0.02170599438250065  val loss:  0.02680838294327259
epoch:  13   step:  67   train loss:  0.028792599216103554  val loss:  0.027594532817602158
epoch:  13   step:  68   train loss:  0.035197436809539795  val loss:  0.027276666834950

epoch:  13   step:  148   train loss:  0.02719808556139469  val loss:  0.029469875618815422
epoch:  13   step:  149   train loss:  0.0258046742528677  val loss:  0.030056729912757874
epoch:  13   step:  150   train loss:  0.03871145844459534  val loss:  0.029495395720005035
epoch:  13   step:  151   train loss:  0.029254725202918053  val loss:  0.02862645499408245
epoch:  13   step:  152   train loss:  0.03233083710074425  val loss:  0.02884840779006481
epoch:  13   step:  153   train loss:  0.016418373212218285  val loss:  0.02867114543914795
epoch:  13   step:  154   train loss:  0.018076850101351738  val loss:  0.028148161247372627
epoch:  13   step:  155   train loss:  0.011179724708199501  val loss:  0.028570465743541718
epoch:  13   step:  156   train loss:  0.021082734689116478  val loss:  0.02971516363322735
epoch:  13   step:  157   train loss:  0.021824944764375687  val loss:  0.030253034085035324
epoch:  13   step:  158   train loss:  0.026822196319699287  val loss:  0.03066

KeyboardInterrupt: 

In [8]:
from torch.utils.tensorboard import SummaryWriter   
import math

writer = SummaryWriter('./net_params_20210503/net_params_20210503_1')

acc = [1.1043323278427124, 0.7861866354942322, 0.33089593052864075, 0.17106975615024567, 0.10820182412862778, 0.06893736869096756, 0.05178064480423927, 0.04006130248308182, 0.031190240755677223, 0.027450941503047943, 0.022916141897439957, 0.022088101133704185, 0.0203436017036438]

print(len(acc))

for epoch in range(len(acc)):
    acc_e = math.exp(-acc[epoch])
    print(acc_e)
    writer.add_scalar('accuracy', acc_e, epoch)


13
0.33143209636728294
0.4555787749628391
0.7182799161622191
0.8427627835338143
0.8974464518032462
0.9333851373805397
0.9495371299032743
0.9607305421792682
0.9692911568526904
0.9729224114938483
0.977344438587109
0.9781540547738128
0.9798619332282431
